In [51]:
import pandas as pd
import numpy as np
from scipy.signal import correlate
from scipy.stats import shapiro
import plotly.express as px
import plotly.graph_objects as go
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense
import statsmodels.api as sm
from statsmodels.api import OLS

pd.set_option('display.max_columns', None)

In [22]:
# Cases data
cases = pd.read_csv("./cases_sold.csv")

# Inventory data
inv = pd.read_csv("./inventory.csv")
inv['BRNCH_CD'] = inv['BRNCH_CD'].astype('string')
inv['TEMP_ZONE'] = inv['TEMP_ZONE'].astype('string')
inv['TEMP_ZONE'] = inv['TEMP_ZONE'].replace(['CLR', 'DRY', 'FRZ'], ['Refrigerated', 'Dry', 'Freezer'])
inv = inv.groupby(['FISC_YR_WK', 'BRNCH_CD']).agg(sum).reset_index()

# Spoilage data
spoilage = pd.read_csv("./spoilage.csv")
spoilage['TEMP_ZONE'] = spoilage['TEMP_ZONE'].replace(['CLR', 'DRY', 'FRZ'], ['Refrigerated', 'Dry', 'Freezer'])
spoilage = spoilage.groupby(['FISC_YR_WK', 'BRNCH_CD']).agg(sum).reset_index()

# Slot utilization data
slot_util = pd.read_csv("./Slot Utilization.csv")
slot_util['DATE_EXTRACT'] = pd.to_datetime(slot_util['DATE_EXTRACT'])

# Remove rows with no capacity and where branches are equal to X1, X6, or X7 and not stock yards
slot_util = slot_util[slot_util['CAPACITY'].notna()]
slot_util = slot_util[~slot_util['BRNCH_CD'].isin(['X1', 'X6', 'X7'])]
slot_util = slot_util[~slot_util['FULL_MARKET_NAME'].str.contains('STOCK YARDS')]
slot_util_main = slot_util[slot_util['CAPACITY'] != 0]

In [23]:
pd.read_csv("./Slot Utilization.csv")


,WAREHOUSE_LOCN,AREA,BRNCH_CD,FULL_MARKET_NAME,STORAGE_TYPE,PICK_TYPE,DATE_EXTRACT,SUM(PALLET_USED),SUM(PALLET_POSITIONS),CAPACITY
0,2125,Freezer,5O,"MANASSAS (5O, 2125)",Main Warehouse,Reserve,2022-05-16,9358.0,13044,0.717417
1,3148,Freezer,4H,"SALT LAKE CITY (4H, 4118)",Off-Site Storage,Reserve,2022-05-03,NaN,0,NaN
2,2345,Refrigerated,3J,"BISMARCK (3J, 2345)",Main Warehouse,Reserve,2021-10-19,477.0,905,0.527071
3,4135,Freezer,8T,"PHOENIX SYSTEMS (8T, 4135)",Main Warehouse,Reserve,2021-10-08,1330.0,1694,0.785123
4,1106,Dry,8L,"DETROIT (8L, 1106)",Main Warehouse,Pick,2021-05-12,5585.0,6243,0.894601
...,...,...,...,...,...,...,...,...,...,...
1138487,4118,Freezer,4H,"SALT LAKE CITY (4H, 4118)",Virtual Slot,Pick,2022-01-28,NaN,0,NaN
1138488,2190,Freezer,2R,"BUFFALO (2R, 2190)",Inactive,Reserve,2022-03-24,NaN,0,NaN
1138489,2190,Refrigerated,2R,"BUFFALO (2R, 2190)",Inactive,Reserve,2022-10-24,NaN,0,NaN
1138490,3310,Freezer,9J,"BILLINGS (9J, 3310)",Inactive,Reserve,2022-09-15,NaN,0,NaN


In [24]:
# Total number of pallet positions used by day and branch 
brnchs = slot_util.groupby(['DATE_EXTRACT', 'BRNCH_CD']).agg(np.sum).reset_index()
brnchs['CAPACITY'] = brnchs['SUM(PALLET_USED)'] / brnchs['SUM(PALLET_POSITIONS)']
brnchs = brnchs[['DATE_EXTRACT', 'BRNCH_CD', 'SUM(PALLET_USED)']]
brnchs = brnchs.rename(columns={'SUM(PALLET_USED)':'TOTAL_PALLETS_USED'})

# Max number and percentage of pallet positions that are virtual slots by week and branch
virt = slot_util.groupby(['DATE_EXTRACT', 'STORAGE_TYPE', 'BRNCH_CD']).agg(np.sum).reset_index()
virt = virt.merge(brnchs, how='left', on=['DATE_EXTRACT', 'BRNCH_CD'])
virt = virt[['DATE_EXTRACT', 'STORAGE_TYPE', 'BRNCH_CD', 'SUM(PALLET_USED)', 'TOTAL_PALLETS_USED']]
virt = virt.rename(columns={'SUM(PALLET_USED)':'VIRTUAL_SLOT_POSITIONS'})
virt = virt[virt['STORAGE_TYPE'] == 'Virtual Slot']
virt['PERC_VIRTUAL_SLOTS'] = virt['VIRTUAL_SLOT_POSITIONS'] / virt['TOTAL_PALLETS_USED']
fw_virt = virt['DATE_EXTRACT'].apply(lambda a : int(str(a.year) + str(a.week)))
virt['FISC_YR_WK'] = fw_virt
virt = virt.groupby(['FISC_YR_WK', 'BRNCH_CD']).agg(max).reset_index()
virt = virt[['FISC_YR_WK', 'BRNCH_CD', 'VIRTUAL_SLOT_POSITIONS', 'PERC_VIRTUAL_SLOTS']]
virt = virt.rename(columns={'VIRTUAL_SLOT_POSITIONS':'MAX_VIRTUAL_SLOT_POSITIONS', 
                            'PERC_VIRTUAL_SLOTS':'MAX_PERC_VIRTUAL_SLOTS'})

# Max number and percentage of slots that are actively being picked by week and branch
pick = slot_util.groupby(['DATE_EXTRACT', 'PICK_TYPE', 'BRNCH_CD']).agg(np.sum).reset_index()
pick = pick.merge(brnchs, how='left', on=['DATE_EXTRACT', 'BRNCH_CD'])
pick = pick[['DATE_EXTRACT', 'PICK_TYPE', 'BRNCH_CD', 'SUM(PALLET_USED)', 'TOTAL_PALLETS_USED']]
pick = pick.rename(columns={'SUM(PALLET_USED)':'PICK_POSITIONS'})
pick = pick[pick['PICK_TYPE'] == 'Pick']
pick['PERC_PICK'] = pick['PICK_POSITIONS'] / pick['TOTAL_PALLETS_USED']
fw_pick = pick['DATE_EXTRACT'].apply(lambda a : int(str(a.year) + str(a.week)))
pick['FISC_YR_WK'] = fw_pick
pick = pick.groupby(['FISC_YR_WK', 'BRNCH_CD']).agg(max).reset_index()
pick = pick[['FISC_YR_WK', 'BRNCH_CD', 'PICK_POSITIONS', 'PERC_PICK']]
pick = pick.rename(columns={'PICK_POSITIONS':'MAX_PICK_POSITIONS', 
                            'PERC_PICK':'MAX_PERC_PICK'})
pick

,FISC_YR_WK,BRNCH_CD,MAX_PICK_POSITIONS,MAX_PERC_PICK
0,20216,2G,8782.0,0.488894
1,20216,2I,15441.0,0.364235
2,20216,2J,10002.0,0.429031
3,20216,2L,4697.0,0.465510
4,20216,2N,2367.0,0.328841
...,...,...,...,...
6979,202352,9L,7133.0,0.458655
6980,202352,9O,4521.0,0.350519
6981,202352,9P,10355.0,0.388031
6982,202352,9Q,8011.0,0.409477


# Updated Merging
### Slot utilization aggregated to weekly first and then merged with other tables afterwards.
### Spoilage data not included because the model performs best without it and we're able to use more data
### One row corresponds to average capacity over the week for a single branch along with cases sold, inventory, and other derived features

In [25]:
# Adding fiscal year and week to slot_util data to allow merging with other tables
fw = slot_util['DATE_EXTRACT'].apply(lambda a : int(str(a.year) + str(a.week)))
slot_util['FISC_YR_WK'] = fw
slot_util = slot_util.groupby(['BRNCH_CD', 'FISC_YR_WK', 'DATE_EXTRACT']).agg(sum).reset_index()
slot_util = slot_util.drop('DATE_EXTRACT', axis=1)
slot_util = slot_util.groupby(['BRNCH_CD', 'FISC_YR_WK']).agg(np.mean).reset_index()
slot_util['CAPACITY'] = slot_util['SUM(PALLET_USED)'] / slot_util['SUM(PALLET_POSITIONS)']

# Merging slot utilization by week and branch
merged = slot_util.merge(cases, how='inner', on=['BRNCH_CD', 'FISC_YR_WK'], validate="m:1")
merged['DIV_NBR'] = merged['DIV_NBR'].fillna(0)

# Merge with inventory data on branch, week, and area
merged = merged.merge(inv, how='inner', left_on=['BRNCH_CD', 'FISC_YR_WK'], 
                      right_on=['BRNCH_CD', 'FISC_YR_WK'], validate="m:1")
merged.head()

# Merge with derived virtual slot features
merged = merged.merge(virt, how='inner', on=['BRNCH_CD', 'FISC_YR_WK'])

# Merge with derived pick slot features
merged = merged.merge(pick, how='inner', on=['BRNCH_CD', 'FISC_YR_WK'])

# Dropping redundant or useless columns
final = merged.drop(['WAREHOUSE_LOCN', 'DIV_NBR', 'DIV_NM', 'SUM(PALLET_USED)', 'SUM(PALLET_POSITIONS)'], axis=1)

In [35]:
print(final.head())
final.to_csv("aggregated_new.csv", index=False)

  BRNCH_CD  FISC_YR_WK  CAPACITY   CASES_SOLD  MAX_WKLY_INVENTORY  \
0       2G      202110  0.832141  125766.5582       504549.312504   
1       2G      202111  0.825156  130699.0329       505326.354171   
2       2G      202112  0.823667  131386.2356       494993.050005   
3       2G      202113  0.825510  136079.6219       497737.160838   
4       2G      202114  0.836580  129391.5811       514711.669166   

   MAX_VIRTUAL_SLOT_POSITIONS  MAX_PERC_VIRTUAL_SLOTS  MAX_PICK_POSITIONS  \
0                        38.0                0.002127              8892.0   
1                        37.0                0.002073              8809.0   
2                        35.0                0.001957              8746.0   
3                        39.0                0.002139              8792.0   
4                        36.0                0.001988              8797.0   

   MAX_PERC_PICK  
0       0.487828  
1       0.490418  
2       0.489869  
3       0.482123  
4       0.479388  


# Updated Cross Correlation Using Weekly Data

In [38]:
temp = final.groupby('FISC_YR_WK').agg(np.sum).reset_index()
x = temp['CAPACITY']
y = temp['CASES_SOLD']
lag = np.arange(-len(x) + 1, len(x))
coefs = correlate(x-x.mean(), y-y.mean(), mode='full') / (np.std(x) * np.std(y) * len(x))
px.line(x=lag, y=coefs, labels={'x':'Lag', 'y':'Correlation Coef'}, 
        title=('Correlation between ' + x.name + ' and ' + y.name),
        range_y=[-1, 1], range_x=[-52, 52])

In [39]:
x = temp['CAPACITY']
y = temp['MAX_WKLY_INVENTORY']
lag = np.arange(-len(x) + 1, len(x))
coefs = correlate(x-x.mean(), y-y.mean(), mode='full') / (np.std(x) * np.std(y) * len(x))
px.line(x=lag, y=coefs, labels={'x':'Lag', 'y':'Correlation Coef'}, 
        title=('Correlation between ' + x.name + ' and ' + y.name),
        range_y=[-1, 1], range_x=[-52, 52])

In [40]:
x = temp['CASES_SOLD']
y = temp['MAX_WKLY_INVENTORY']
lag = np.arange(-len(x) + 1, len(x))
coefs = correlate(x-x.mean(), y-y.mean(), mode='full') / (np.std(x) * np.std(y) * len(x))
px.line(x=lag, y=coefs, labels={'x':'Lag', 'y':'Correlation Coef'}, 
        title=('Correlation between ' + x.name + ' and ' + y.name),
        range_y=[-1, 1], range_x=[-24, 24])

# Model Building

In [41]:
x = pd.get_dummies(final.drop(['CAPACITY', 'FISC_YR_WK'], axis=1))
y = final['CAPACITY']
x

,CASES_SOLD,MAX_WKLY_INVENTORY,MAX_VIRTUAL_SLOT_POSITIONS,MAX_PERC_VIRTUAL_SLOTS,MAX_PICK_POSITIONS,MAX_PERC_PICK,BRNCH_CD_2G,BRNCH_CD_2I,BRNCH_CD_2J,BRNCH_CD_2L,BRNCH_CD_2N,BRNCH_CD_2O,BRNCH_CD_2R,BRNCH_CD_2Z,BRNCH_CD_3D,BRNCH_CD_3F,BRNCH_CD_3J,BRNCH_CD_3K,BRNCH_CD_3L,BRNCH_CD_3M,BRNCH_CD_3V,BRNCH_CD_3W,BRNCH_CD_3Y,BRNCH_CD_3Z,BRNCH_CD_4C,BRNCH_CD_4H,BRNCH_CD_4I,BRNCH_CD_4J,BRNCH_CD_4O,BRNCH_CD_4P,BRNCH_CD_4Q,BRNCH_CD_4R,BRNCH_CD_4U,BRNCH_CD_4V,BRNCH_CD_5D,BRNCH_CD_5E,BRNCH_CD_5G,BRNCH_CD_5I,BRNCH_CD_5O,BRNCH_CD_5T,BRNCH_CD_5Y,BRNCH_CD_5Z,BRNCH_CD_6A,BRNCH_CD_6B,BRNCH_CD_6F,BRNCH_CD_6G,BRNCH_CD_6H,BRNCH_CD_6I,BRNCH_CD_6J,BRNCH_CD_6N,BRNCH_CD_6U,BRNCH_CD_6W,BRNCH_CD_6Y,BRNCH_CD_6Z,BRNCH_CD_8A,BRNCH_CD_8B,BRNCH_CD_8E,BRNCH_CD_8L,BRNCH_CD_8N,BRNCH_CD_8O,BRNCH_CD_8S,BRNCH_CD_8T,BRNCH_CD_8U,BRNCH_CD_8V,BRNCH_CD_9A,BRNCH_CD_9B,BRNCH_CD_9D,BRNCH_CD_9I,BRNCH_CD_9J,BRNCH_CD_9L,BRNCH_CD_9O,BRNCH_CD_9P,BRNCH_CD_9Q,BRNCH_CD_9U
0,125766.5582,504549.312504,38.0,0.002127,8892.0,0.487828,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,130699.0329,505326.354171,37.0,0.002073,8809.0,0.490418,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,131386.2356,494993.050005,35.0,0.001957,8746.0,0.489869,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,136079.6219,497737.160838,39.0,0.002139,8792.0,0.482123,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,129391.5811,514711.669166,36.0,0.001988,8797.0,0.479388,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4950,158822.0846,519287.973613,44.0,0.002536,7986.0,0.445841,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4951,165377.2505,526855.627776,53.0,0.002809,7905.0,0.435032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4952,162917.1169,542519.291667,41.0,0.002260,7927.0,0.428469,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4953,133906.8864,563538.537500,49.0,0.002624,8035.0,0.425718,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
# Train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=12345)

# Scoring method
scorer = make_scorer(mean_squared_error)

# PCA parameters
components = None

# Baseline linear regression model
pipe = Pipeline([('robust scaling', RobustScaler(with_centering=False)),
                 #('pca', PCA(n_components=components)),
                 ('lm', LinearRegression())])
pipe.fit(x_train, y_train)
res = cross_validate(pipe, x_test, y_test, cv=5, scoring=scorer)
print(np.mean(res['test_score']))

5.544248716717709e+19


In [43]:
# Baseline linear regression model
fit = sm.OLS(y_train, x_train).fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               CAPACITY   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     755.1
Date:                Fri, 24 Mar 2023   Prob (F-statistic):               0.00
Time:                        08:51:48   Log-Likelihood:                 8930.6
No. Observations:                3716   AIC:                        -1.771e+04
Df Residuals:                    3642   BIC:                        -1.725e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
CASES_SOLD                  3.702e-08   1.69e-08      2.196      0.028    3.96e-09    7.01e-08
MAX_WKLY_INVENTORY          2.655e-08   4.77e-09      5.570      0.000    1.72e-08    3.59e-08
MAX_VIRTUAL_SLOT_POSITIONS -8.637e-06   9.49e-06     -0.910      0.363   -2.72e-05    9.96e-06
MAX_PERC_VIRTUAL_SLOTS         0.9066      0.256      3.542      0.000       0.405       1.408
MAX_PICK_POSITIONS          4.824e-05   9.04e-07     53.376      0.000    4.65e-05       5e-05
MAX_PERC_PICK                 -1.6836      0.026    -64.260      0.000      -1.735      -1.632
BRNCH_CD_2G                    1.1776      0.011    104.580      0.000       1.156       1.200
BRNCH_CD_2I                    0.5641      0.015     38.753      0.000       0.536       0.593
BRNCH_CD_2J                    0.9485      0.011     84.130      0.000       0.926       0.971
BRNCH_CD_2L                    1.3018      0.011    122.596      0.000       1.281       1.323
BRNCH_CD_2N                    1.1077      0.008    141.499      0.000       1.092       1.123
BRNCH_CD_2O                    0.7747      0.012     64.284      0.000       0.751       0.798
BRNCH_CD_2R                    1.2976      0.011    116.815      0.000       1.276       1.319
BRNCH_CD_2Z                    1.1150      0.011    105.137      0.000       1.094       1.136
BRNCH_CD_3D                    1.3305      0.012    111.094      0.000       1.307       1.354
BRNCH_CD_3F                    0.9483      0.014     70.142      0.000       0.922       0.975
BRNCH_CD_3J                    1.7088      0.017    102.288      0.000       1.676       1.742
BRNCH_CD_3K                    1.2959      0.015     84.657      0.000       1.266       1.326
BRNCH_CD_3L                    1.5172      0.014    105.084      0.000       1.489       1.546
BRNCH_CD_3M                    0.7152      0.011     67.234      0.000       0.694       0.736
BRNCH_CD_3V                    1.1021      0.011    101.455      0.000       1.081       1.123
BRNCH_CD_3W                    1.0981      0.011    100.632      0.000       1.077       1.120
BRNCH_CD_3Y                    1.0262      0.012     87.439      0.000       1.003       1.049
BRNCH_CD_3Z                    1.2287      0.011    109.637      0.000       1.207       1.251
BRNCH_CD_4C                    0.7400      0.013     55.018      0.000       0.714       0.766
BRNCH_CD_4H                    1.0408      0.011     95.557      0.000       1.019       1.062
BRNCH_CD_4I                    0.9360      0.011     84.532      0.000       0.914       0.958
BRNCH_CD_4J                    1.3169      0.012    111.689      0.000       1.294       1.340
BRNCH_CD_4O                    0.8288      0.013     63.882      0.000       0.803       0.854
BRNCH_CD_4P                    0.9342      

In [44]:
# Random forest model
rf = Pipeline([('robust scaling', RobustScaler(with_centering=False)),
                 #('pca', PCA(n_components=components)),
                 ('lm', RandomForestRegressor())])
rf.fit(x_train, y_train)
res = cross_validate(rf, x_test, y_test, cv=5, scoring=scorer)
print(np.mean(res['test_score']))

0.0006649788836285429


In [48]:
# Boosted trees model
boosted = Pipeline([('robust scaling', RobustScaler(with_centering=False)),
                 ('pca', PCA(n_components=components)),
                 ('lm', GradientBoostingRegressor(n_estimators=1500))])
boosted.fit(x_train, y_train)
res = cross_validate(boosted, x_test, y_test, cv=5, scoring=scorer) # seed this later
print(sqrt(np.mean(res['test_score'])))

0.00035616306462263054


In [54]:
print(np.sqrt(np.mean(res['test_score'])))

0.01887228297325553


In [55]:
# MLP
mlp = Sequential()
mlp.add(Dense(32, activation='relu', input_shape=(x_train.shape[1],)))
mlp.add(Dense(16, activation='relu'))
mlp.add(Dense(1, activation='sigmoid'))
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mse')
mlp.fit(x_train, y_train, batch_size=32, epochs=5, verbose=1)
res = mlp.evaluate(x_test, y_test, batch_size=32)
print(res)
print(mlp.predict(np.array(x_test.head())))

Epoch 1/5
117/117 [==============================] - 1s 6ms/step - loss: 0.0873
Epoch 2/5
117/117 [==============================] - 1s 6ms/step - loss: 0.0536
Epoch 3/5
117/117 [==============================] - 1s 6ms/step - loss: 0.0536
Epoch 4/5
117/117 [==============================] - 1s 6ms/step - loss: 0.0536
Epoch 5/5
39/39 [==============================] - 0s 4ms/step - loss: 0.0549
0.054896727204322815
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [46]:
# Hyperparameter tuning for SVM model
# parameters = {'gamma':['scale', 'auto'], 'C':[0.5, 1, 2], 'degree':[1, 2, 3, 4], 'kernel':['poly', 'rbf', 'sigmoid']}
parameters = {'C':[0.5, 1, 2], 'kernel':['rbf'], 'gamma':['scale', 'auto']}
gs = GridSearchCV(SVR(), parameters, n_jobs=4)
gs.fit(x_train, y_train)

GridSearchCV(estimator=SVR(), n_jobs=4,
             param_grid={'C': [0.5, 1, 2], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf']})

In [47]:
# Evaluating best SVM model on test set 
pipe = Pipeline([('robust scaling', RobustScaler(with_centering=False)),
                 #('pca', PCA(n_components=components)),                 
                 ('svm', SVR(C=gs.best_params_['C'],
                             kernel=gs.best_params_['kernel'],
                             gamma=gs.best_params_['gamma']))])
pipe.fit(x_train, y_train)
res = cross_validate(pipe, x_test, y_test, cv=5, scoring=scorer)
print(np.mean(res['test_score']))

0.003692924452433968
